# Objective

Off label use with Tolvaptan (Jinarc) to treat infant population (up to 2 yrs) with a severe potentially fatal polycystosis kidney disease, which parents presented with, possibly asymptomatic, ADPKRD or ADPKD

## Details
- Total ADPKD in data set
- % on drug tovalptan
- Median/mean age
- Insurance type 
- And any other relevant descriptives.

## Scoping essentials:
For scoping however all we want to establish is that there are some patients in our dataset with ADPKD who have used tovalptan. 

## ICD code notes from Joe
Using ICD codes:  ('q611','q6119','75313') I found only 280 total pharmacy claims in Condor (all years, no patient limitations).  

Using NDC codes: ('46602002105','75839016905','46602002005','59148002050','59148002150') for Tolvaptan /Jinarc / JYNARQUE/ SAMSCA (US version I believe) I found 2,592 claims for 2017.

## Adding more ICD codes?
Ref: https://dailymed.nlm.nih.gov/dailymed/search.cfm?query=tolvaptan 
NDC codes: 591480087007, 591480087028, 591480088007, 591480088028, 591480089007, 591480089028, 591480020050, 591480021050 



# Nish

In [5]:
%%read_sql

select *
from RWD_DB.RWD.AMN_REDBOOK_LATEST
where gennme ilike '%tolvaptan%'

Query started at 12:10:13 PM GMT Daylight Time; Query executed in 0.11 m

,ndcnum,deaclas,generid,maintain,orgbkcd,prodcat,siglsrc,genind,orgbkfg,desidrg,...,thrdtds,mstfmds,thrclds,thrgrds,deaclds,prodnme,manfnm,gennme,roacd,roads
0,59148002050,6,127439,3,NR,1,Y,1,N,N,...,Tolvaptan,Tablet,"Misc Therapeutic Agents, NEC",Misc Therapeutic Agents,All other Rx,SAMSCA,"OTSUKA AMERICA PHARMACEUTICALS, INC",Tolvaptan,PO,Oral
1,59148002150,6,127440,3,NR,1,Y,1,N,N,...,Tolvaptan,Tablet,"Misc Therapeutic Agents, NEC",Misc Therapeutic Agents,All other Rx,SAMSCA,"OTSUKA AMERICA PHARMACEUTICALS, INC",Tolvaptan,PO,Oral


# Initialization

## Importing relevant modules

In [2]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Connecting to snowflake

In [3]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Reference table

## ICD grouper table for reference

In [4]:
snow.select("select * from RWD_DB.RWD.ICD_GROUPER limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,1,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A00,No map,No map,Cholera,Cholera,2017-11-02,2017-11-02
1,2,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A000,0010,Cholera due to vibrio cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,2017-11-02,2017-11-02
2,3,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A001,0011,Cholera due to vibrio cholerae el tor,Cholera due to Vibrio cholerae 01 biovar eltor,Cholera due to Vibrio cholerae 01 biovar eltor,2017-11-02,2017-11-02


## Checking for ADPKD codes

In [6]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%polycystic%'
    and level_4_short_description_icd10 ilike '%kidney%'
    --and level_4_short_description_icd10 ilike '%disease%'

Query started at 04:35:13 PM India Standard Time; Query executed in 0.06 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,28033,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q611,No map,No map,Polycystic kidney infantile type,Polycystic kidney infantile type,2017-11-02,2017-11-02
1,28035,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q6119,75314,Polycystic kidney autosomal recessive,Other polycystic kidney infantile type,Other polycystic kidney infantile type,2017-11-02,2017-11-02
2,28036,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q612,75313,Polycystic kidney autosomal dominant,Polycystic kidney adult type,Polycystic kidney adult type,2017-11-02,2017-11-02
3,28037,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q613,75312,Polycystic kidney unspecified type,Polycystic kidney unspecified,Polycystic kidney unspecified,2017-11-02,2017-11-02
4,93591,Z00 - Z9989,Factors influencing health status and contact ...,Z77-Z99,Persons with potential health hazards related ...,Z82-Z828,Fam hx of certain disabil & chr dis (leading t...,Z8271,V1861,Family history of polycystic kidney,Family history of polycystic kidney,Family history of polycystic kidney,2017-11-02,2017-11-02


## Checking pharmacy records for tovalptan

In [7]:
snow.select("select * from RWD_DB.RWD.RAVEN_PHARMACY limit 3")

,amount_attributed_to_sales_tax,amount_exceeding_periodic_benefit_maximum,amount_of_copay_or_coinsurance,basis_of_cost_determination,basis_of_reimbursement_determination,bin_number,cardholder_id,carrier_id,claim_number,claim_or_reference_id,...,unit_of_measure,vendor_id,version_number,date_of_birth,zip3,data_source,load_indicator,inferred_payer_1,inferred_payer_2,inferred_payer_3
0,None,None,39.0,01,03,610011,None,None,3055040809,None,...,EA,1,D0,1945-01-01,342,CONDOR,H,677,NaN,NaN
1,None,None,45.0,01,03,610502,None,None,3054031436,None,...,None,1,D0,1940-01-01,000000000000000,CONDOR,H,6,NaN,NaN
2,None,None,3.0,00,13,600428,None,None,3055170065,None,...,EA,1,D0,1974-01-01,563,CONDOR,H,250,251.0,252.0


## Relevant columns from pharmacy table
CLAIM_NUMBER,DIAGNOSIS_CODE, date_of_birth, ENCRYPTED_KEY_1, ENCRYPTED_KEY_2, NCPDP_NUMBER, PATIENT_GENDER_CODE, UNIT_DOSE_INDICATOR, DATE_OF_SERVICE, DATE_PRESCRIPTION_WRITTEN

## Restricting to diagnosis codes only

In [28]:
%%read_sql
create or replace table st_pharma_cohort_a as
select
left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
left(date_of_service, 4) as service_year,
left(DATE_PRESCRIPTION_WRITTEN, 4) as prescription_year,
claim_number,
diagnosis_code,
date_of_birth,
PRODUCT_OR_SERVICE_ID,
PATIENT_GENDER_CODE, 
UNIT_DOSE_INDICATOR

from
RWD_DB.RWD.RAVEN_PHARMACY
where
diagnosis_code in ('Q611','Q6119', 'Q612', 'Q613', '75314', '75313', '75314')
--and PRODUCT_OR_SERVICE_ID in ('5914808707', '5914808728', '5914808807', '5914808828', '5914808907', '5914808928', '5914802050', '5914802150')
--and date_of_service > '2015-05-31'
--and date_of_service < '2017-07-01'

Query started at 08:24:29 PM India Standard Time; Query executed in 8.18 m

,status
0,Table ST_PHARMA_COHORT_A successfully created.


In [29]:
snow.select("select count(distinct claim_number) from st_pharma_cohort_a")

,COUNT(DISTINCT CLAIM_NUMBER)
0,0


In [25]:
snow.select("select count(distinct patient_id) from st_pharma_cohort_a")

,COUNT(DISTINCT PATIENT_ID)
0,1214


## Restricting to diagnosis codes and NDC codes

In [30]:
%%read_sql
create or replace table st_pharma_cohort1 as
select
left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
left(date_of_service, 4) as service_year,
left(DATE_PRESCRIPTION_WRITTEN, 4) as prescription_year,
claim_number,
diagnosis_code,
date_of_birth,
PRODUCT_OR_SERVICE_ID,
PATIENT_GENDER_CODE, 
UNIT_DOSE_INDICATOR

from
RWD_DB.RWD.RAVEN_PHARMACY
where
diagnosis_code in ('Q611','Q6119', 'Q612', 'Q613', '75314', '75313', '75314')
and PRODUCT_OR_SERVICE_ID in ('46602002105','75839016905','46602002005','59148002050','59148002150')

Query started at 08:39:42 PM India Standard Time; Query executed in 14.38 m

,status
0,Table ST_PHARMA_COHORT1 successfully created.


In [31]:
snow.select("select count(distinct claim_number) from st_pharma_cohort1")

,COUNT(DISTINCT CLAIM_NUMBER)
0,31


# Checking EHR for ADPKD patients

In [3]:
snow.select("select * from RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


,claim_number,encrypted_key_1,encrypted_key_2,procedure_type,procedure,procedure_sequence,line_number,statement_from,statement_to,service_from,...,hglbn_rslt_gm_nbr,hmtcr_rslt_pct,acct_lab_npi_id,eptin_strt_dsg_nbr,crtin_rslt_nbr,upc_cde,data_source,load_indicator,etl_create_ts,etl_update_ts
0,con_EI120516289348214,XSkNeyg2ZLg8vJSgFwHMkugmfFvuoTL8ReNaXdgwnTc=,iWIbh+5zDaMB5O35wDzKYPXsB64cdkgN70OekjDu/4w=,CPT,50590,5,5,2016-11-11,2016-11-11,2016-11-11,...,None,None,None,None,None,None,CONDOR,H,2017-01-12,2017-01-12
1,con_EP111716737613189,pEOriKSmf+uLGEgzKNJ0RK0zMWk6nIq5brxpMQnvyQQ=,eBjmgkZRGIseWctSwaVG/ehM/RTUJeQY7MGBPiTmSDE=,CPT,50590,1,1,None,None,2016-11-11,...,None,None,None,None,None,None,CONDOR,H,2016-12-12,2016-12-12
2,con_EI121316219749249,i98xUdfxiW8wD7WkmIgKnVpJhnPujvEM1KxM/6mt0pQ=,FsEAUG4K4vBqCnAMHt6jw5QE9b1q3ZvzjbqNq4dC+Q0=,CPT,50590,6,6,2016-11-11,2016-11-11,2016-11-11,...,None,None,None,None,None,None,CONDOR,H,2017-01-17,2017-01-17


In [5]:
%%read_sql
create or replace table st_pharma_cohort as
select
left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
YEAR_OF_SERVICE,
claim_number,
diagnosis

from
RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS

where
diagnosis in ('Q611','Q6119', 'Q612', 'Q613', '75314', '75313', '75314')
--and PRODUCT_OR_SERVICE_ID in ('46602002105','75839016905','46602002005','59148002050','59148002150')

Query started at 03:53:54 PM India Standard Time; Query executed in 6.50 m

,status
0,Table ST_PHARMA_COHORT successfully created.


# Checking EHR: Albatross for diagnosis

In [9]:
snow.select("select * from RWD_DB.RWD.albatross_EHR_patients limit 3")

,rectype,rectypeversion,genclientid,gen2clientid,genpatientid,gen2patientid,dobyear,deceasedflag,gender,race,...,state_x,smokingstatusflag,lastupdatedttm,genproviderid,gen2providerid,primarykey,patient_suffix,encrypted_key_1,encrypted_key_2,tier_identifier
0,PAT,5.2,20720,D7B7161B8096C012C4EFC4E0B27273D050D0303B535CE0...,145185657,0D8E6809FA73C8F84E21B2668890B5A1D50E62BE9D9E04...,1965,0,Female,None,...,PA,None,None,4984481,D0BB2DC442ED5DEB1812B3A9837DCFA63CFFCCE779EA57...,428EDA58-820B-4A02-8AD8-944C1F9EE851,7LJ5dGdLZR84hZRfw8pO9KV1rzy8U10wYBcj6s5VNUc=,7LJ5dGdLZR84hZRfw8pO9KV1rzy8U10wYBcj6s5VNUc=,XixEJ9dl/fIWQUL4qif+zcNKupzXURCm4WDkN9zkbk0=,Tier2
1,PAT,5.2,20711,C03185E9171B2F3E6CE1F6A76C92625C939B7D4701711D...,142221131,1E7FED9712ABE75DCD5CD348E7D318EE32698D14383EF2...,1948,0,Female,White,...,GA,None,None,0,None,3C2FCDB6-23FB-4B98-AF02-DE9AF1D41A9E,Qw1Fs/ZtlV4mja2iNzjykRC+srABsqecRPrKwtS29qg=,Qw1Fs/ZtlV4mja2iNzjykRC+srABsqecRPrKwtS29qg=,1F8QVicTEgaF2mV5usIyznRLKrsJ9BA5Dai8JEjSmNU=,Tier2
2,PAT,5.2,20720,D7B7161B8096C012C4EFC4E0B27273D050D0303B535CE0...,145073547,3A8D1BFD9330CB35DEACFB77CE7363990CC85DB90D46F4...,1928,0,Male,None,...,PA,None,None,4984499,E9460F07A3B784F21E68615D27A5DEA43676C8BB3B21B4...,A17A279A-71EB-404E-A950-3EB7172A1065,+CzFkC0M2mFaa39nrB2Ay6Ck1eGEyVgCV535cyeVOhk=,+CzFkC0M2mFaa39nrB2Ay6Ck1eGEyVgCV535cyeVOhk=,8M1kcecDuge2Lk1aoX3284ID1VfPGfutnJ3lhZMq9Qg=,Tier2


In [15]:

%%read_sql
create or replace table st_adpkd1 as
select 
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    icd9,
    HIPAA_ICD9,
    icd10,
    HIPAA_ICD10,
    recordeddttm,
    left(recordeddttm, 4) as year
    
from RWD_DB.RWD.albatross_EHR_problems
where 
   icd10 in ('Q611','Q6119', 'Q612', 'Q613')
or ICD9 in ('75314', '75313', '75314')

Query started at 04:36:36 PM India Standard Time; Query executed in 3.70 m

,status
0,Table ST_ADPKD1 successfully created.


## Grouping albatross information

In [16]:
snow.select ("select count (distinct patient_id) from st_adpkd1")

,COUNT (DISTINCT PATIENT_ID)
0,20530


# Raven medical claims
Using medical claims because of better coverage and completeness of records

## Fetching relevant diagnosis patients from the diagnosis table
('Q611','Q6119', 'Q612', 'Q613', '75314', '75313', '75314')

In [17]:
%%read_sql
create or replace table ST_ADPKD_diag as
select
left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
claim_number,
diagnosis,
diagnosis_sequence,
year_of_service
from
RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
diagnosis in ('Q611','Q6119', 'Q612', 'Q613', '75314', '75313', '75314')
and year_of_service > '2016-12-31'
and year_of_service < '2018-01-01'

Query started at 04:47:40 PM India Standard Time; Query executed in 8.89 m

,status
0,Table ST_ADPKD_DIAG successfully created.


In [18]:
snow.select ("select count (distinct patient_id) from ST_ADPKD_diag")

,COUNT (DISTINCT PATIENT_ID)
0,61885


## Fetch patients matching the diag/claim in diag table

In [21]:
%%read_sql
create or replace table ST_ADPKD_pts as
select
'XXXXXXXXXXXXXXXX' as patient_id,
claim_number,
patient_gender,
patient_dob
from RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PATIENT 
where patient_dob > '2014-12-31'

Query started at 05:07:33 PM India Standard Time; Query executed in 2.43 m

,status
0,Table ST_ADPKD_PTS successfully created.


In [22]:
%%read_sql
begin;
update ST_ADPKD_diag a
set a.patient_id = b.patient_id
from ST_ADPKD_pts b
where a.claim_number = b.claim_number;
commit;

Query started at 05:10:31 PM India Standard Time; Query executed in 0.05 mQuery started at 05:10:34 PM India Standard Time; Query executed in 0.10 mQuery started at 05:10:40 PM India Standard Time; Query executed in 0.04 m

,status
0,Statement executed successfully.


In [23]:
%%read_sql
select
count(patient_id),
count(distinct patient_id),
sum(case
when patient_id = 'XXXXXXXXXXXXXXXX'
then 1
else 0
end) as null_pt_id,
sum(case
when left(patient_id, 5) = 'XXX -'
then 1
else 0
end) as faulty_pt_id
from ST_ADPKD_diag

Query started at 05:11:25 PM India Standard Time; Query executed in 0.05 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,223518,61120,4070,4353


## Pharmacy table relevant columns

In [25]:
%%read_sql
create or replace table ST_ADPKD_PHARMA as
select
'XXXXXXXXXXXXXXXX' as patient_id,
claim_number,
PRODUCT_OR_SERVICE_ID

from RWD_DB.RWD.RAVEN_PHARMACY

where claim_number in (select distinct claim_number from ST_ADPKD_diag)
and PRODUCT_OR_SERVICE_ID in ('46602002105','75839016905','46602002005','59148002050','59148002150')

Query started at 05:26:31 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 2.55 m

,status
0,Table ST_ADPKD_PHARMA successfully created.


## Updating diagnosis table

In [26]:
%%read_sql
begin;
update ST_ADPKD_diag a
set a.patient_id = b.patient_id
from ST_ADPKD_PHARMA b
where a.claim_number = b.claim_number;
commit;

Query started at 05:30:38 PM India Standard Time; Query executed in 0.04 mQuery started at 05:30:41 PM India Standard Time; Query executed in 0.05 mQuery started at 05:30:44 PM India Standard Time; Query executed in 0.14 m

,status
0,Statement executed successfully.


In [27]:
%%read_sql
select
count(patient_id),
count(distinct patient_id),
sum(case
when patient_id = 'XXXXXXXXXXXXXXXX'
then 1
else 0
end) as null_pt_id,
sum(case
when left(patient_id, 5) = 'XXX -'
then 1
else 0
end) as faulty_pt_id
from ST_ADPKD_diag

Query started at 05:30:53 PM India Standard Time; Query executed in 0.08 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,223518,61120,4070,4353


## Final counts

In [28]:
snow.select ("select * from ST_ADPKD_diag limit 5")

,patient_id,claim_number,diagnosis,diagnosis_sequence,year_of_service
0,uKjXyaManSvAYplx,295698000000204945,Q613,1,2017-09-30
1,WuyNuKiItsqe9Sai,con_EP062718764334324,Q613,2,2017-07-24
2,Vw+aqQVqoTyJp6RZ,295625000000414235,Q613,6,2017-10-29
3,SO6IBInaXZAPT1C1,vul_95d41e26-2786-44e8-bc95-2fd31e6034fd,Q612,2,2017-04-13
4,sqyPWk2GogYuxxCE,vul_cbafea1f-cb67-4df1-beea-7c40d57ca5cf,Q613,2,2017-04-02


In [30]:
snow.select ("select * from ST_ADPKD_PTS limit 5")

,patient_id,claim_number,patient_gender,patient_dob
0,XXXXXXXXXXXXXXXX,vul_591167965,M,2017-01-01
1,XXXXXXXXXXXXXXXX,con_EP061316715177348,M,2016-01-01
2,XXXXXXXXXXXXXXXX,con_EP021717735411680,F,2017-01-01
3,XXXXXXXXXXXXXXXX,con_EP122215743506481,M,2015-01-01
4,XXXXXXXXXXXXXXXX,con_EI062515271173154,M,2015-01-01


In [31]:
%%read_sql
create or replace table ST_ADPKD_PHARMA as
select
left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
claim_number,
PRODUCT_OR_SERVICE_ID

from RWD_DB.RWD.RAVEN_PHARMACY

where claim_number in (select distinct claim_number from ST_ADPKD_diag)
and PRODUCT_OR_SERVICE_ID in ('46602002105','75839016905','46602002005','59148002050','59148002150')

Query started at 05:36:57 PM India Standard Time; Query executed in 12.47 m

,status
0,Table ST_ADPKD_PHARMA successfully created.


In [32]:
snow.select ("select count (distinct patient_id) from ST_ADPKD_PHARMA")

,COUNT (DISTINCT PATIENT_ID)
0,0


In [33]:
%%read_sql
create or replace table ST_ADPKD_PHARMA1 as
select
left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
claim_number,
PRODUCT_OR_SERVICE_ID

from RWD_DB.RWD.RAVEN_PHARMACY

where PRODUCT_OR_SERVICE_ID in ('46602002105','75839016905','46602002005','59148002050','59148002150')

Query started at 05:54:20 PM India Standard Time; Query executed in 6.46 m

,status
0,Table ST_ADPKD_PHARMA1 successfully created.


In [35]:
snow.select ("select count (distinct patient_id) from ST_ADPKD_PHARMA1")

,COUNT (DISTINCT PATIENT_ID)
0,6692
